In [26]:
#path_wet="/data5/eloi/TencyMastering/part1/11172/dry_multi/1110_s_Drums_multi.wav"
#path_dry="/data5/eloi/TencyMastering/part1/11172/multi/1110_s_Drums_multi.wav"
#alignment_file="/data5/eloi/TencyMastering/part1/11172/alignment.pickle"

path_wet="/data5/eloi/TencyMastering/part3/66537/dry_multi/1110_Drums_multi.wav"
path_dry="/data5/eloi/TencyMastering/part3/66537/multi/1110_Drums_multi.wav"
alignment_file="/data5/eloi/TencyMastering/part3/66537/alignment.pickle"

import soundfile as sf
import numpy as np
import os
import sys
import torch

y, fs= sf.read(path_wet)
y = torch.tensor(y.T).float() # Add batch dimension

x, fs = sf.read(path_dry)
x = torch.tensor(x.T).float()  # Add batch dimension

import pickle
alignment_data = pickle.load(open(alignment_file, "rb"))


def read_wav_segment(file_path, start=None, end=None, dtype="float32"):
    """
    Reads a specific segment from a .wav file efficiently.

    Args:
        file_path (str): Path to the .wav file.
        start (int): Start frame index.
        end (int): End frame index.

    Returns:
        numpy.ndarray: Audio data for the specified segment.
        int: Sample rate of the audio file.
    """
    # Open the .wav file
    if start is None or end is None:
        data, samplerate = sf.read(file_path, dtype=dtype)
    else:
        with sf.SoundFile(file_path) as audio_file:
            # Read only the required frames
            audio_file.seek(start)
            data = audio_file.read(frames=end-start, dtype=dtype)
            samplerate = audio_file.samplerate

    return data, samplerate
def load_audio( file, start=None, end=None, stereo=True):

            x, fs=read_wav_segment(file, start, end)
            if stereo:
                if len(x.shape)==1:
                    #print( "dry not stereo , doubling channels", x_dry.shape)
                    x=x[:,np.newaxis]
                    x= np.concatenate((x, x), axis=-1)
                elif len(x.shape)==2 and x.shape[-1]==1:
                    #print( "dry not stereo , doubling channels", x_dry.shape)
                    x = np.concatenate((x, x), axis=-1)

            x=torch.from_numpy(x).permute(1,0)

            return x, fs

start=1674158
end=start+84100
#start_dry=start+alignment_data["dry_alignment"]
#end_dry=end+alignment_data["dry_alignment"]
#
#start_wet=start+alignment_data["multi_alignment"]
#end_wet=end+alignment_data["multi_alignment"]
#y_segment, fs = load_audio(path_wet, start=start_wet, end=end_wet, stereo=True)
#x_segment, fs = load_audio(path_dry, start=start_dry, end=end_dry, stereo=True)

#print(f"wet segment shape: {y_segment.shape}, dry segment shape: {x_segment.shape}, fs: {fs}")

def efficient_roll(x, shift, dims=-1):
    """
    Efficiently roll tensor elements along a dimension without creating a full copy.
    
    Args:
        x: Input tensor
        shift: Number of places to roll (negative for left roll)
        dim: Dimension along which to roll
    
    Returns:
        Rolled tensor view where possible, minimal copy where necessary
    """
    if shift == 0:
        return x
    
    # Get the size of the dimension
    dim_size = x.size(dims)
    
    # Handle shift larger than dimension size
    shift = shift % dim_size
    if shift < 0:
        shift += dim_size
    
    # Create indices for the roll
    indices = torch.cat([torch.arange(dim_size-shift, dim_size), 
                         torch.arange(0, dim_size-shift)])
    
    # Use index_select for the roll
    return torch.index_select(x, dims, indices)

x_aligned=efficient_roll(x, -alignment_data["dry_alignment"], dims=-1)
y_aligned=efficient_roll(y, -alignment_data["multi_alignment"], dims=-1)

x_segment=x_aligned[:, start:end]
y_segment=y_aligned[:, start:end]





In [27]:
import plotly.express as px
start=x.shape[-1]//8 + 800000
end=start+44100
shift_x= -110245
shift_y= 11
#x_segment=x[..., (start+shift_x):(end+shift_x)]
#y_segment=y[..., (start+shift_y):(end+shift_y)]


#calculate the crosscorrelation between the two segments
crosscorr= torch.nn.functional.conv1d(
    y_segment.unsqueeze(0), x_segment.unsqueeze(0), padding="same"
).squeeze(0)
print(crosscorr)


fig=px.line( title="Dry Segment")
fig.add_scatter(y=x_segment[0].numpy(), mode="lines", name="dry Segment")
fig.add_scatter(y=y_segment[0].numpy(), mode="lines", name="Wet Segment")
fig.update_layout(xaxis_title="Time (samples)", yaxis_title="Amplitude")
fig.show()

tensor([[-0.0218, -0.0327, -0.0211,  ...,  0.0222,  0.0308,  0.0273]])


In [15]:
start

1674158

In [7]:
crosscorr.shape

torch.Size([1, 44100])

In [17]:
px.line(crosscorr[0].numpy(), title="Cross-correlation").show()